In [ ]:
#!/usr/bin/env python3
"""
测试和配置黑体字体用于matplotlib中文显示
"""

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib as mpl
import numpy as np
import subprocess
import os

def find_heiti_fonts():
    """查找系统中的黑体字体"""
    print("=== 查找黑体字体 ===\n")
    
    # 系统中的常见黑体字体
    heiti_keywords = [
        'hei', '黑体', 'microsoft', 'wenquan', 'ukai', 'simhei', 
        'microsoft yahei', 'pingfang', 'hiragino', 'source han'
    ]
    
    font_paths = fm.findSystemFonts()
    heiti_fonts = []
    
    print("扫描系统字体...")
    for font_path in font_paths:
        try:
            font_prop = fm.FontProperties(fname=font_path)
            font_name = font_prop.get_name()
            
            # 检查是否包含黑体关键词
            if any(keyword in font_name.lower() for keyword in heiti_keywords):
                heiti_fonts.append((font_name, font_path))
                print(f"  发现黑体字体: {font_name}")
                print(f"    路径: {font_path}")
                
        except Exception as e:
            continue
    
    if not heiti_fonts:
        print("❌ 未找到黑体字体")
        print("尝试安装文泉驿黑体...")
        install_heiti_fonts()
        return find_heiti_fonts()  # 重新扫描
    
    return heiti_fonts

def install_heiti_fonts():
    """安装黑体字体"""
    print("正在安装黑体字体...")
    
    try:
        # 安装文泉驿黑体
        commands = [
            ['sudo', 'apt-get', 'update'],
            ['sudo', 'apt-get', 'install', '-y', 'fonts-wqy-zenhei'],
            ['sudo', 'apt-get', 'install', '-y', 'fonts-wqy-microhei'],
            ['fc-cache', '-fv']
        ]
        
        for cmd in commands:
            result = subprocess.run(cmd, capture_output=True, text=True)
            if result.returncode != 0:
                print(f"命令失败: {' '.join(cmd)}")
                print(f"错误: {result.stderr}")
            else:
                print(f"✅ {' '.join(cmd)}")
                
    except Exception as e:
        print(f"安装失败: {e}")

def test_heiti_fonts(heiti_fonts):
    """测试黑体字体的中文显示"""
    print("\n=== 测试黑体字体 ===\n")
    
    working_fonts = []
    
    for font_name, font_path in heiti_fonts:
        print(f"测试字体: {font_name}")
        
        try:
            # 设置字体
            mpl.rcParams['font.sans-serif'] = [font_name, 'DejaVu Sans']
            mpl.rcParams['axes.unicode_minus'] = False
            
            # 创建测试图
            fig, ax = plt.subplots(figsize=(8, 6))
            
            # 测试数据
            x = np.linspace(0, 5, 50)
            y = np.sin(x)
            
            ax.plot(x, y, 'b-', linewidth=2)
            ax.set_title('黑体中文标题测试')
            ax.set_xlabel('横坐标轴')
            ax.set_ylabel('纵坐标轴')
            ax.grid(True, alpha=0.3)
            
            # 保存测试图像
            test_file = f'heiti_test_{font_name.replace(" ", "_")}.png'
            fig.savefig(test_file, dpi=150, bbox_inches='tight')
            plt.close(fig)
            
            print(f"  ✅ 成功: {test_file}")
            working_fonts.append((font_name, font_path))
            
        except Exception as e:
            print(f"  ❌ 失败: {e}")
    
    return working_fonts

def create_heiti_config(working_fonts):
    """创建黑体字体配置"""
    if not working_fonts:
        print("❌ 没有可用的黑体字体")
        return
    
    # 使用第一个可用的黑体字体
    best_font_name, best_font_path = working_fonts[0]
    
    print(f"\n🎯 推荐使用字体: {best_font_name}")
    print(f"字体路径: {best_font_path}")
    
    # 创建配置文件
    config_dir = os.path.expanduser('~/.config/matplotlib')
    os.makedirs(config_dir, exist_ok=True)
    
    config_content = f"""# matplotlib 黑体中文配置
font.family: sans-serif
font.sans-serif: {best_font_name}, DejaVu Sans, Arial Unicode MS
axes.unicode_minus: False
"""
    
    config_file = os.path.join(config_dir, 'matplotlibrc')
    with open(config_file, 'w', encoding='utf-8') as f:
        f.write(config_content)
    
    print(f"✅ 配置文件已更新: {config_file}")
    print("配置内容:")
    print(config_content)
    
    # 测试配置
    test_final_config(best_font_name)

def test_final_config(font_name):
    """测试最终配置"""
    print("\n=== 最终配置测试 ===\n")
    
    # 重新加载配置
    mpl.rcParams['font.sans-serif'] = [font_name, 'DejaVu Sans']
    mpl.rcParams['axes.unicode_minus'] = False
    
    # 创建综合测试图
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle('黑体中文显示测试', fontsize=16, fontweight='bold')
    
    # 子图1: 线图
    x = np.linspace(0, 10, 50)
    y1 = np.sin(x)
    y2 = np.cos(x)
    axes[0,0].plot(x, y1, 'b-', linewidth=2, label='正弦波')
    axes[0,0].plot(x, y2, 'r--', linewidth=2, label='余弦波')
    axes[0,0].set_title('三角函数曲线')
    axes[0,0].set_xlabel('横坐标')
    axes[0,0].set_ylabel('纵坐标')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 子图2: 柱状图
    categories = ['苹果', '香蕉', '橙子', '葡萄']
    values = [25, 30, 20, 15]
    bars = axes[0,1].bar(categories, values, color=['red', 'yellow', 'orange', 'purple'])
    axes[0,1].set_title('水果销量统计')
    axes[0,1].set_ylabel('销量 (kg)')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # 子图3: 饼图
    sizes = [40, 30, 20, 10]
    labels = ['产品A', '产品B', '产品C', '其他']
    colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99']
    axes[1,0].pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    axes[1,0].set_title('市场份额分布')
    axes[1,0].axis('equal')
    
    # 子图4: 散点图
    x_scatter = np.random.normal(50, 10, 30)
    y_scatter = np.random.normal(60, 8, 30)
    axes[1,1].scatter(x_scatter, y_scatter, alpha=0.7, c='blue', edgecolors='black')
    axes[1,1].set_title('数据点分布')
    axes[1,1].set_xlabel('特征X')
    axes[1,1].set_ylabel('特征Y')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('heiti_chinese_final_test.png', dpi=150, bbox_inches='tight', 
                facecolor='white', edgecolor='none')
    plt.show()
    
    print("✅ 最终测试图像已保存: heiti_chinese_final_test.png")
    print("请检查图像中的中文是否正常显示为黑体")

if __name__ == "__main__":
    # 查找黑体字体
    heiti_fonts = find_heiti_fonts()
    
    if heiti_fonts:
        # 测试黑体字体
        working_fonts = test_heiti_fonts(heiti_fonts)
        
        if working_fonts:
            # 创建配置
            create_heiti_config(working_fonts)
        else:
            print("❌ 没有可用的黑体字体，尝试安装...")
            install_heiti_fonts()
    else:
        print("❌ 未找到黑体字体，跳过测试")

SyntaxError: invalid syntax (2310308833.py, line 233)